# Baseball ETL

### Imports

In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from config import pg_pwd
import psycopg2

from bs4 import BeautifulSoup as bs
import requests

import pandas as pd
import numpy as np

### Initializations

In [2]:
#db initializations
engine = create_engine(f'postgresql://{pg_pwd}/MoneyBall')

Base = automap_base()
Base.prepare(autoload_with=engine)
session = Session(engine)

### ETL Baseball Cities

In [3]:
# initialize
City = Base.classes.City

# extract
url = 'https://www.worldatlas.com/articles/mlb-teams-and-their-cities.html'

response = requests.get(url)
soup = bs(response.text, 'html.parser')
result = soup.find('tbody')
rows = result.find_all('tr')

In [4]:
# transfer and load

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]   
    cols = cols[2].split(', ')
    
    city = City(cityname=cols[0], statename=cols[1])  
    session.add(city)
    session.commit()

In [5]:
# ETL Baseball Teams

In [6]:
# ETL Baseball Players

In [7]:
# ETL Baseball Player Salaries

In [8]:
# ETL Baseball Player Batting

In [9]:
# ETL Baseball Player Pitching

In [10]:
# ETL Baseball Player Cards

In [11]:
# ETL Baseball Team Patroll

In [13]:
session.close()
engine.dispose()